### Notebook para analisar os corpora PetroNER

In [34]:
from conllu import parse_incr, parse
from owlready2 import *

Carregando os corpora e a ontologia

In [40]:
# CONLLU parser

# Corpora completo
PetroNER = open("../PetroNER/petroner-uri-2022-03-07.conllu", "r", encoding="utf-8")
PetroNER_sentences = parse_incr(PetroNER)

# Corpora treino
PetroNER_treino = open("../PetroNER/petroner-Treino-uri-2022-03-07.conllu", "r", encoding="utf-8")
PetroNER_treino_sentences = parse_incr(PetroNER_treino)

# Corpora validação
PetroNER_valid = open("../PetroNER/petroner-Validação-uri-2022-03-07.conllu", "r", encoding="utf-8")
PetroNER_valid_sentences = parse_incr(PetroNER_valid)

# Corpora teste
PetroNER_teste = open("../PetroNER/petroner-Teste-uri-2022-03-07.conllu", "r", encoding="utf-8")
PetroNER_teste_sentences = parse_incr(PetroNER_teste)

In [41]:
#OntoGeoLógica populada (OntoGeoLogicaPopulada.owl)
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaPovoadaInstanciasRelacoes.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

Função que recebe o texto anotado em formato CONLLU e processa para o treinamento. Tem como saída as sentenças acrescida das tags para indicar as entidades, bem como com a entidade no início do texto; lista com as entidades (instâncias extraídas do texto); lista com as classes que as entidades pertencem; e lista com as URI referente às entidades. O texto sai com o seguinte formato:  
"[Nome da classe] Nome da entidade | blablabla blablabla blablabla [E] Nome da entidade [/E] blablabla blablabla." 

In [42]:
# Função que recebe
def preparar_dataset(tokenlist):

    #sent = TokenList([])
    sent_orig = ''   # String para compor a sentença original
    sents_NER = []   # Lista para armazenar as sentenças com indicação das entidades
    NER_on = False   # Variável para indicar se está iterando por token de entidades
    NER_tok = ''     # string para armazenar os tokens das entidades que estão sendo iteradas
    MW_on = False    # Variável para indicar se está iterando por tolkens multiword
    MW_tok = ''      # string para armazenar o termo multiword
    MW_end = 0       # ID final do token multiword
    ENT = []         # Lista para armazenar as entidades encontradas no texto
    ENT_class = []   # Lista para armazenar as classes das entidades encontradas no texto
    ENT_URI = []     # Lista para armazenar as URI das entidades encontradas no texto

    #Iterando por todos os tokens de uma sentença
    for tok in tokenlist:
        # ignorando id não inteiro (tuplas), ou seja token multiword
        if type(tok['id']) == int:

            # Verifica se o token inicia uma entidade ("B")
            if tok['deps'][0] == 'B':
                
                # Verifica se a variável NER_on está ativado (No caso de um tok "B" seguido de outro)
                if NER_on == True:
                    
                    NER_on = False  # Desativa a variável NER_on

                    # Para sentença mais recente da lista sents_NER, inclui a entidade no início e o marcador [/E] no final 
                    sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                    # Incluir o NER_tok na lista ENT
                    ENT.append(NER_tok.strip())

                    NER_tok = '' # Apaga a string NER_tok

                NER_on = True  # Ativa a variável NER_on
                # Acrescenta uma sentença com o marcador [E] na listas de sentenças com NER
                sents_NER.append(sent_orig + ' [E]')
                # Incluir classe na lista ENT_class
                ENT_class.append(tok['deps'][2:])
                # Incluir URI da entidade na lista ENT_URI. Caso não haja URI, incluir "None". 
                try:
                    ENT_URI.append(tok['misc']['grafo'])
                except:
                    ENT_URI.append(None)

            # Identifica o primeiro token "O" após uma sequência de entidades
            if (tok['deps'][0] == 'O') & NER_on:
                
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok
                
            # Identifica se o último token da sequência tem o marcador "B" ou "I".    
            if (tokenlist[-1]['id'] == tok['id']) & ((tok['deps'][0] == 'B') | (tok['deps'][0] == 'I')):
                NER_on = False  # Desativa a variável NER_on

                # Para sentença mais recente da lista sents_NER, 
                # inclui a classe e a entidade no início e o marcador [/E] no final 
                sents_NER[-1] = '[' + ENT_class[-1] + '] ' + NER_tok + '| ' + sents_NER[-1].strip() + ' [/E]'

                # Incluir o NER_tok na lista ENT
                ENT.append(NER_tok.strip())

                NER_tok = '' # Apaga a string NER_tok

            # Verifica se está iterando por uma multiword ou não
            if not MW_on:

                # Acrecenta o token à string da entidade
                if NER_on:
                    NER_tok = NER_tok + tok['form'] + ' '

                # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                if tok['upos'] == 'PUNCT':
                    # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                    sent_orig = sent_orig + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + tok['form']
                else:
                    sent_orig = sent_orig + ' ' + tok['form']
                    for i in range(len(sents_NER)):
                        sents_NER[i] = sents_NER[i] + ' ' + tok['form']

            else:
                # Se o token for o último termo da multiword, desativamos a variável MW_on
                if tok['id'] == MW_end:
                    MW_on = False

                    # Acrecenta a multiword à string da entidade
                    if NER_on:
                        NER_tok = NER_tok + MW_tok + ' '

                    #Ao invés de incluir nas senteças o token, vamos incluir o MW_tok 
                    # Se o token for uma pontuação, não inclui espaço entre o token anterior 
                    if tok['upos'] == 'PUNCT':
                        # O token é incluído na senteça original e em todas as sentenças da lista sents_NER
                        sent_orig = sent_orig + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + MW_tok
                    else:
                        sent_orig = sent_orig + ' ' + MW_tok
                        for i in range(len(sents_NER)):
                            sents_NER[i] = sents_NER[i] + ' ' + MW_tok

        # Se a ID do token for referente a uma multiword, ativamos a variável MW_on, 
        # armazenamos o termo MW_tok e o valor da 'id' final
        else:
            MW_on = True
            MW_tok = tok['form']
            MW_end = tok['id'][-1]
    
    return(sents_NER, ENT, ENT_class, ENT_URI)

Função que itera por todas as sentneças de um corpora e retorna as listas com as sentenças, entidades, classes e URI

In [43]:
def ent_class_URI(sentences):

    # Separando as listas de sentenças, entidades, classes e URI
    text = []
    entities = []
    classes_NER = [] 
    ENT_URI = []
    entities_sem_URI = []

    # Iterando por todas as sentenças em formato CONLLU para preparar o dataset
    n = 0
    m = 0
    
    for tokenlist in sentences:
        # contador de sentenças
        n = n + 1
        # processando cada sentenças
        text_n, entities_n, classes_NER_n, ENT_URI_n = preparar_dataset(tokenlist)
        # Contador de sentenãs com entidades
        if len(text_n) > 0:
            m = m + 1
        
        text = text + text_n
        entities = entities + entities_n
        classes_NER = classes_NER + classes_NER_n
        ENT_URI = ENT_URI + ENT_URI_n

    print('Total de sentenças no arquivo: ', n)
    print('N° de sentenças com entidades: ', m, 'que equivale a ', (m/n)*100, '% do total')
    print ('N° de entidades: ', len(entities))
    print('N° de classes: ', len(classes_NER))
    print('Média de ', len(entities)/m, 'entidades por sentença (das sentenças que possuem entidades)')
    
    #Verificando entidades sem URI
    for i in range(len(ENT_URI)):
        if ENT_URI[i] == None:
            entities_sem_URI.append(entities[i])
        
    print('N° de entities com URI: ', len(ENT_URI) - len(entities_sem_URI))
    print('N° de entities sem URI: ', len(entities_sem_URI))
    print((len(entities_sem_URI)/len(ENT_URI))*100, '% das entidades não possuem URI.')
    
    return (text, entities, classes_NER, ENT_URI, entities_sem_URI)

In [44]:
print('PetroNER')
PetroNER_text, PetroNER_entities, PetroNER_classes_NER, PetroNER_ENT_URI, PetroNER_entities_sem_URI = ent_class_URI(PetroNER_sentences)

print('\nPetroNER - Treino')
treino_text, treino_entities, treino_classes_NER, treino_ENT_URI, treino_entities_sem_URI = ent_class_URI(PetroNER_treino_sentences)

print('\nPetroNER - Validação')
valid_text, valid_entities, valid_classes_NER, valid_ENT_URI, valid_entities_sem_URI = ent_class_URI(PetroNER_valid_sentences)

print('\nPetroNER - Teste')
teste_text, teste_entities, teste_classes_NER, teste_ENT_URI, teste_entities_sem_URI = ent_class_URI(PetroNER_teste_sentences)

PetroNER
Total de sentenças no arquivo:  24035
N° de sentenças com entidades:  8757 que equivale a  36.434366548783025 % do total
N° de entidades:  18749
N° de classes:  18749
Média de  2.141030033116364 entidades por sentença (das sentenças que possuem entidades)
N° de entities com URI:  13102
N° de entities sem URI:  5647
30.118939676782762 % das entidades não possuem URI.

PetroNER - Treino
Total de sentenças no arquivo:  17987
N° de sentenças com entidades:  6526 que equivale a  36.28175904820148 % do total
N° de entidades:  13904
N° de classes:  13904
Média de  2.1305547042598834 entidades por sentença (das sentenças que possuem entidades)
N° de entities com URI:  9642
N° de entities sem URI:  4262
30.653049482163407 % das entidades não possuem URI.

PetroNER - Validação
Total de sentenças no arquivo:  2467
N° de sentenças com entidades:  892 que equivale a  36.15727604377787 % do total
N° de entidades:  1754
N° de classes:  1754
Média de  1.9663677130044843 entidades por sentença

In [26]:
PetroNER_entities_sem_URI[:10]

['Santana',
 'Membro Mucuri',
 'hidrocarbonetos',
 'selo',
 'petróleo',
 'água',
 'poços',
 'petróleo',
 'poços',
 'poços']